In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division

In [2]:
import sys
import argparse
import numpy as np
from scipy.stats import norm
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
%matplotlib inline

sns.set(color_codes=True)

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [1]:
import tensorflow as tf
import numpy as np
from ops import *
import sampler

In [2]:
def generator(start_depth, out_dim=[32, 32], reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    
    
    z = tf.placeholder(dtype=tf.float32, shape=[None, 100])
    D = start_depth
    
    h0 = linear(z, out_dim=D * np.prod(out_dim), name='projection')
    h0 = tf.reshape(h0, [-1, out_dim[0]//16, out_dim[1]//16, D])
    
    h1 = conv_T(h0, kernel_size=5, depth=D//2, name='conv1')
    h1 = tf.nn.relu(h1)
    
    h2 = conv_T(h1, kernel_size=5, depth=D//4, name='conv2')
    h2 = tf.nn.relu(h2)
    
    h3 = conv_T(h2, kernel_size=5, depth=D//8, name='con3')
    h3 = tf.nn.relu(h3)
    
    G = conv_T(h3, kernel_size=5, depth=3, name='conv4')
    G = tf.nn.relu(G)
    
    return G

In [3]:
def discriminator(start_depth, im_dim=[32, 32, 3], reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    
    im = tf.placeholder(dtype=tf.float32, shape=[None] + im_dim)
    D = start_depth
    
    h0 = conv(im, kernel_size=5, depth=D, name='conv1')
    h0 = lrelu(tf.nn.bh0)
    
    h1 = conv(h0, kernel_size=5, depth=D*2, name='conv2')
    h1 = lrelu(h1)
    
    h2 = conv(h2, kernel_size=5, depth=D*4, name='conv3')
    h2 = lrelu(h2)
    
    h3 = conv(h1, kernel_size=5, depth=D*8, name='conv4')
    h3 = lrelu(h3)
    
    h4 = linear(tf.reshape(h3, [None, -1]), out_dim=1)
    
    D = tf.nn.sigmoid(h4, name='linear')
    
    return D

In [4]:
with tf.variable_scope('G') as scope:
    G = generator(512)

In [5]:
for v in tf.trainable_variables(): print(v)

Tensor("G/projection/W/read:0", shape=(100, 524288), dtype=float32)
Tensor("G/projection/b/read:0", shape=(524288,), dtype=float32)
Tensor("G/conv1/K/read:0", shape=(5, 5, 256, 512), dtype=float32)
Tensor("G/conv1/b/read:0", shape=(256,), dtype=float32)
Tensor("G/conv2/K/read:0", shape=(5, 5, 128, 256), dtype=float32)
Tensor("G/conv2/b/read:0", shape=(128,), dtype=float32)
Tensor("G/con3/K/read:0", shape=(5, 5, 64, 128), dtype=float32)
Tensor("G/con3/b/read:0", shape=(64,), dtype=float32)
Tensor("G/conv4/K/read:0", shape=(5, 5, 3, 64), dtype=float32)
Tensor("G/conv4/b/read:0", shape=(3,), dtype=float32)


In [7]:
with tf.variable_scope('D') as scope:
    D = generator(512)

In [8]:
for v in tf.trainable_variables(): print(v)

Tensor("G/projection/W/read:0", shape=(100, 524288), dtype=float32)
Tensor("G/projection/b/read:0", shape=(524288,), dtype=float32)
Tensor("G/conv1/K/read:0", shape=(5, 5, 256, 512), dtype=float32)
Tensor("G/conv1/b/read:0", shape=(256,), dtype=float32)
Tensor("G/conv2/K/read:0", shape=(5, 5, 128, 256), dtype=float32)
Tensor("G/conv2/b/read:0", shape=(128,), dtype=float32)
Tensor("G/con3/K/read:0", shape=(5, 5, 64, 128), dtype=float32)
Tensor("G/con3/b/read:0", shape=(64,), dtype=float32)
Tensor("G/conv4/K/read:0", shape=(5, 5, 3, 64), dtype=float32)
Tensor("G/conv4/b/read:0", shape=(3,), dtype=float32)
Tensor("D/projection/W/read:0", shape=(100, 524288), dtype=float32)
Tensor("D/projection/b/read:0", shape=(524288,), dtype=float32)
Tensor("D/conv1/K/read:0", shape=(5, 5, 256, 512), dtype=float32)
Tensor("D/conv1/b/read:0", shape=(256,), dtype=float32)
Tensor("D/conv2/K/read:0", shape=(5, 5, 128, 256), dtype=float32)
Tensor("D/conv2/b/read:0", shape=(128,), dtype=float32)
Tensor("D/con